In [ ]:
import sys

sys.path.insert(0, "../")
sys.path.insert(0, "packages")

In [ ]:
import os

if os.environ.get("CIRCLECI"):
    default_env = os.environ.get("CONDA_DEFAULT_ENV")
    os.environ["PYSPARK_DRIVER_PYTHON"] = (
        f"/home/circleci/miniconda/envs/{default_env}/bin/python"
    )
    os.environ["PYSPARK_PYTHON"] = (
        f"/home/circleci/miniconda/envs/{default_env}/bin/python"
    )

# Time Series Sub-Module

This tutorial walks through some of the basics of the time series submodule, which
leverages Spark's higher-order-functions to perform calculations, converting the
problem from tall format to wide format.

See [the tall vs wide experiment](./10_timeseries_experiment.md) for an analysis on
performance of slicing (window function equivalent) at scale.


A few other points:

* If you are not familar with higher-order-functions, you can follow Databrick's tutorial
[here](https://docs.databricks.com/delta/data-transformation/higher-order-lambda-functions.html).
* In Spark 3.1, we can write actual python functions rather than SQL strings. See
[here](https://towardsdatascience.com/higher-order-functions-with-spark-3-1-7c6cf591beaa).
* Here is an interesting article that benchmarks different approaches and compares the performance
of higher-order-functions: [article](https://towardsdatascience.com/performance-in-apache-spark-benchmark-9-different-techniques-955d3cc93266).

## Motivation

When it comes to analysing timeseries data or any data that requires ordering such as
sensor data, it is believed that tall format might not be the most ideal data structure
for such problems. This is because one cannot assume the data is sorted without first
explicitly ordering the data, especially when it comes to spark where the underlying
data is stored in various files. Example below when we read data from file:

```
# data could be this
| hcp_id | time_index | value |
|--------|------------|-------|
| h1     | 1          | 10    |
| h1     | 2          | 3     |
| h1     | 3          | 4     |
| h1     | 4          | 2     |
| h1     | 5          | 7     |
| h1     | 6          | 2     |
| h2     | 1          | 4     |
| h2     | 2          | 7     |
| h2     | 3          | 2     |
| h2     | 4          | 8     |
| h2     | 5          | 2     |
| h2     | 6          | 9     |

# or this
| hcp_id | time_index | value |
|--------|------------|-------|
| h1     | 1          | 10    |
| h1     | 6          | 2     |
| h1     | 3          | 4     |
| h1     | 4          | 2     |
| h1     | 5          | 7     |
| h1     | 2          | 3     |
| h2     | 6          | 9     |
| h2     | 2          | 7     |
| h2     | 1          | 4     |
| h2     | 4          | 8     |
| h2     | 5          | 2     |
| h2     | 3          | 2     |
```

Additionally, in spark, even if you sort the data, writing to file and re-reading does not preserve
this row ordering. One has to explicitly re-order the data before performing any operation
in order to guarantee that order is respected. When chaining multiple operations together,
each operation still needs to explicitly order and cannot assume the data is ordered from
the operation prior. This leads to inefficiencies in a big data setting as the data
needs to be constantly shuffled and explicitly ordered each time a calculation is performed.
Note that shuffling data is arguably one of the most expensive operations in spark.

In the timeseries module, the main data structure is leveraging arrays, where order
between rows are always preserved. The idea is to utilize the ordering to make calculations
(that require ordering) more efficient because there is no need to explicitly order the
data each time, which in spark may trigger a shuffle:

```
| hcp_id | time_index_array | value_array    |
|--------|------------------|----------------|
| h1     | [1,2,3,4,5,6]    | [10,3,4,2,7,2] |
| h2     | [1,2,3,4,5,6]    | [4,7,2,8,2,9]  |
```

The array structure also has another benefit, which is to allow more complicated calculations
to be performed on ordered data, as one can easily use `udf`s, or distribute any `sklearn`
function over a cluster, or leverage spark's higher-order-function capabilities. Stateful calculations
are easy to write when one's input is a list, as one can iterate through the list in a
normal for-loop and store a state that is modified along the way. While window functions
let's you perform a few basic stateful calculations, they fall short when something more
complicated is needed, for example, performing exponential smoothing or discovering seasonality.

To summarize, the purpose of this submodule is to:

* leverage array data structures for performance gains on ordered data
* enable more complicated calculations when required

## Example Usage

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.config("spark.ui.showConsoleProgress", False).getOrCreate()
import datetime

import pyspark.sql.functions as f

from pyspark.sql.types import (
    IntegerType,
    StringType,
    StructField,
    StructType,
    TimestampType,
)

from feature_generation.v1.core.timeseries.array_collect import (
    collect_array_then_interpolate,
)
from feature_generation.v1.core.timeseries.array_transform import array_smooth_ts_values


schema = StructType(
    [
        StructField("element_id", StringType(), True),
        StructField("reading_ts", TimestampType(), True),
        StructField("reading_val", IntegerType(), True),
    ]
)

data = [
    ("line1", datetime.datetime(1970, 1, 1, 5, 0), 1),
    ("line1", datetime.datetime(1970, 1, 1, 5, 2), 2),
    ("line1", datetime.datetime(1970, 1, 1, 5, 6), 3),
    ("line1", datetime.datetime(1970, 1, 1, 5, 7), 4),
    ("line1", datetime.datetime(1970, 1, 1, 5, 8), 5),
    ("line2", datetime.datetime(1970, 1, 1, 7, 1), 3),
    ("line2", datetime.datetime(1970, 1, 1, 7, 5), 1),
    (
        "line2",
        datetime.datetime(1970, 1, 1, 7, 10),
        8,
    ),
    (
        "line2",
        datetime.datetime(1970, 1, 1, 7, 11),
        6,
    ),
    (
        "line2",
        datetime.datetime(1970, 1, 1, 7, 12),
        7,
    ),
]
mock_datetime_df = spark.createDataFrame(data, schema)

For this demonstration, we will be using the following dataset:

In [ ]:
mock_datetime_df.show(truncate=False)

When dealing with time, we recommend adding a time index. In this case, we will be
adding a minute index. But depending on the problem, you might want to add an hour index,
day index, second index and so on. There are several time index creation functions that
are already available, so we simply import the `minute_index` function and use it:

In [ ]:
from feature_generation.v1.core.timeseries.datetime import minute_index

mock_datetime_df = mock_datetime_df.withColumn(
    "minute_index", minute_index("reading_ts")
)
mock_datetime_df.show(truncate=False)

Next, we need to collect the values in tall format, and convert them to arrays (wide
format):

In [ ]:
from feature_generation.v1.core.timeseries.array_collect import (
    collect_array_then_interpolate,
)
from feature_generation.v1.core.timeseries.array_transform import (
    interpolate_constant,
    scipy_interpolate,
)

collected_array_df = collect_array_then_interpolate(
    df=mock_datetime_df,
    order="minute_index",
    values=["reading_val"],
    groupby="element_id",
    spine="spine",
    interpolate_func=interpolate_constant,
)

collected_array_df.show(truncate=False)

The spine for each row is generated from the min and max values in the original array.
Having a complete timeseries without gaps is crucial when it comes to analysing timeseries
data. This means we can apply most calculations without worry of any gaps, and allows
us to easily downsample when we need to.


The next step is to interpolate the values and fill in the missing values from the spine.
In this case, we simply want to forward fill:

In [ ]:
from feature_generation.v1.core.timeseries.array_transform import scipy_interpolate

collected_array_df = collected_array_df.withColumn(
    "interpolated",
    scipy_interpolate(
        spine_index="spine",
        time_index="minute_index",
        value="reading_val",
        kind="previous",
    ),
)

collected_array_df.show(truncate=False)

Note that the interpolate is a UDF and uses `scipy`'s [interp1d](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html)
function. This may not be the best implementation, but it certainly prevents a lot of
re-inventing the wheel for now.

Once we have our complete signal, we can start to extract information out of it.

There are 2 main types of functions within the time series module:

* `array_aggregate` - receives a vector and returns a scalar
* `array_transform` - receives a vector and returns another vector


Here is an example of how to extract the min and max within the arrays:

In [ ]:
from feature_generation.v1.core.timeseries.array_aggregate import (
    array_max,
    array_min,
)

features = collected_array_df.withColumn(
    "feature_array_max", array_max(value="interpolated")
).withColumn("feature_array_min", array_min(value="interpolated"))

features.show(truncate=False)

Or if we want to smooth the values (the equivalent of taking the average within a
window):

In [ ]:
from feature_generation.v1.core.timeseries.array_transform import array_smooth_ts_values


features = collected_array_df.withColumn(
    "smoothed", array_smooth_ts_values(value="interpolated", length=3)
)

features.show(truncate=False)

### Aggregate over slice
Aggregate over slice method uses wide format instead of traditional long format (group by, agg over window etc.).

Input:

In [ ]:
from pyspark import Row

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
import datetime

import pyspark.sql.functions as f


df_spine = spark.createDataFrame(
    [
        Row(
            npi_id="h1",
            time_index=1,
        ),
        Row(
            npi_id="h1",
            time_index=2,
        ),
        Row(
            npi_id="h1",
            time_index=3,
        ),
        Row(
            npi_id="h1",
            time_index=4,
        ),
        Row(
            npi_id="h1",
            time_index=5,
        ),
        Row(
            npi_id="h1",
            time_index=6,
        ),
        Row(
            npi_id="h1",
            time_index=7,
        ),
        Row(
            npi_id="h1",
            time_index=8,
        ),
        Row(
            npi_id="h1",
            time_index=9,
        ),
        Row(
            npi_id="h1",
            time_index=10,
        ),
        Row(
            npi_id="h1",
            time_index=11,
        ),
    ]
)

df_input_slice = spark.createDataFrame(
    [
        Row(
            npi_id="h1",
            time_index=1,
            value=0,
        ),
        Row(
            npi_id="h1",
            time_index=2,
            value=1,
        ),
        Row(
            npi_id="h1",
            time_index=3,
            value=2,
        ),
        Row(
            npi_id="h1",
            time_index=4,
            value=3,
        ),
        Row(
            npi_id="h1",
            time_index=7,
            value=6,
        ),
        Row(
            npi_id="h1",
            time_index=8,
            value=7,
        ),
        Row(
            npi_id="h1",
            time_index=10,
            value=9,
        ),
        Row(
            npi_id="h1",
            time_index=11,
            value=10,
        ),
    ]
)
print("Input dataframe")
df_input_slice.show(truncate=False)

print("Spine dataframe")
df_spine.show(truncate=False)

Here, as a prerequisite, all the input values and the corresponding time component should be collected in lists in separate columns.

There are a few ways to do that:

1. Aggregation over window.
Pre-requisite for this method is to join the dataframe with spine and fill nulls wherever the values are missing.

In [ ]:
from pyspark.sql import Window

w = (
    Window.partitionBy("npi_id")
    .orderBy("time_index")
    .rangeBetween(Window.unboundedPreceding, Window.unboundedFollowing)
)
df_input_slice_tall = df_spine.join(
    df_input_slice, ["npi_id", "time_index"], "left"
).fillna(0, subset=["value"])

df_collect_list = df_input_slice_tall.withColumn(
    "time_list", f.collect_list("time_index").over(w)
).withColumn("value_list", f.collect_list("value").over(w))

df_collect_list.show(truncate=False)

Downside to this method is that the spark might start getting out of memory if working on really large dataset
because the data is being replicated for every row.

2. Use `groupby` and `aggregate` using `sorted_collect_set` - all of which are performed under
`collect_array_then_interpolate` (see source code if curious).

This is currently the recommended way as this is shown to be more performant based on the testing performed.

In [ ]:
from pyspark.sql import Window
from feature_generation.v1.core.timeseries.array_transform import (
    interpolate_constant,
)


w = Window.partitionBy("npi_id")
list_of_tags = ["value"]

df_collect_list_grp = collect_array_then_interpolate(
    df=df_input_slice,
    order="time_index",
    values=list_of_tags,
    groupby="npi_id",
    spine="spine_index",
    interpolate_func=interpolate_constant,
)

# Join with spine to get to the spine granularity.
df_collect_list_2 = df_spine.join(
    df_collect_list_grp.drop("time_index"), "npi_id", "left"
).select("npi_id", "time_index", "spine_index", "value_array_padded")

df_collect_list_2.show(truncate=False)

For each row, time index acts as the anchor and its corresponding index is searched in time_index_array. Based on this anchor index and the window lower bound and upper bound, the value array is sliced and required aggregation is applied on the sliced array.

In [ ]:
from feature_generation.v1.core.timeseries.array_aggregate import (
    aggregate_over_slice,
    array_sum,
)

df_result = df_collect_list.select(
    "*",
    aggregate_over_slice(
        input_col="value_list",
        lower_bound=-1,
        upper_bound=3,
        anchor="time_index",
        anchor_array="time_list",
        func=array_sum,
        alias="sum_val",
    ),
)


df_result.show(truncate=False)

## Summary of Functions

Below is a summary of the available functions within the sub-module:

In [ ]:
from types import FunctionType
from feature_generation.v1.core import timeseries
import pkgutil


pkgname = timeseries.__name__
pkgpath = timeseries.__path__[0]
found_packages = list(pkgutil.iter_modules([pkgpath], prefix="{}.".format(pkgname)))
importer = found_packages[0][0]
sub_packages = [x.split(".")[-1] for _, x, _ in found_packages]

func_row = []
for idx, _name in enumerate(sub_packages):
    module_spec = importer.find_spec(found_packages[idx][1])
    module = module_spec.loader.load_module(found_packages[idx][1])

    list_of_tags = [
        x
        for x in dir(module)
        if isinstance(getattr(module, x), FunctionType) and not x.startswith("_")
    ]

    from tabulate import tabulate

    for x in list_of_tags:
        x_doc = getattr(module, x).__doc__.split("\n")[0]
        func_row.append(
            {"sub_module": module.__name__, "function": x, "description": x_doc}
        )

import pandas as pd

table = pd.DataFrame(func_row)
print(tabulate(table, headers=table.columns, tablefmt="psql"))